<a href="https://colab.research.google.com/github/LorenzoZaccagnini/cryptography-works/blob/master/Create_MD5_collision_ANSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collisioni MD5

Notebook creato da [Lorenzo Zaccagnini](https://www.linkedin.com/in/lorenzo-zaccagnini/)

Questa è l'immagine del progetto dell'ANSA che usa MD5
![alt text](https://www.cryptoavvocato.it/wp-content/uploads/2020/04/ANSA-Ceck.jpg)

Il problema è l'uso del MD5 come identificativo univoco, in questo notebook verrà dimostrato sia possibile creare lo stesso MD5 con due binari differenti.

[Lo script è preso da questa repository](https://github.com/sadhikar/MD5Collision)

[Il paper](https://www.mscs.dal.ca/~selinger/md5collision/) 

[Altro esempio dove si hanno due immagini uguali con lo stesso MD5](https://natmchugh.blogspot.com/2014/10/how-i-created-two-images-with-same-md5.html)

1. Scarichiamo la repository dentro colab, ci permetterà di creare due programmi con istruzioni differenti ma con lo stesso MD5


In [1]:
!wget https://github.com/sadhikar/MD5Collision/archive/master.zip

--2020-04-14 16:49:41--  https://github.com/sadhikar/MD5Collision/archive/master.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/sadhikar/MD5Collision/zip/master [following]
--2020-04-14 16:49:42--  https://codeload.github.com/sadhikar/MD5Collision/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [  <=>               ] 159.01K   287KB/s    in 0.6s    

2020-04-14 16:49:43 (287 KB/s) - ‘master.zip’ saved [162829]



In [2]:
!unzip master.zip

Archive:  master.zip
d3cb502ca8b072d49c7484e2af981596fc4cce2b
   creating: MD5Collision-master/
   creating: MD5Collision-master/MD5collision_Output/
  inflating: MD5Collision-master/MD5collision_Output/evil  
  inflating: MD5Collision-master/MD5collision_Output/good  
  inflating: MD5Collision-master/README.md  
   creating: MD5Collision-master/evilize-0.1/
  inflating: MD5Collision-master/evilize-0.1/COPYING  
  inflating: MD5Collision-master/evilize-0.1/MBSD-LICENSE  
  inflating: MD5Collision-master/evilize-0.1/Makefile  
  inflating: MD5Collision-master/evilize-0.1/README  
  inflating: MD5Collision-master/evilize-0.1/crib.h  
  inflating: MD5Collision-master/evilize-0.1/evil  
  inflating: MD5Collision-master/evilize-0.1/evilize  
  inflating: MD5Collision-master/evilize-0.1/evilize.c  
  inflating: MD5Collision-master/evilize-0.1/evilize.o  
  inflating: MD5Collision-master/evilize-0.1/good  
  inflating: MD5Collision-master/evilize-0.1/goodevil.c  
  inflating: MD5Collision-mas

In [3]:
%cd MD5Collision-master/evilize-0.1/

/content/MD5Collision-master/evilize-0.1


In [4]:
%pwd

'/content/MD5Collision-master/evilize-0.1'

Seguendo [i passaggi descritti nella repo originale](https://github.com/sadhikar/MD5Collision/tree/master/evilize-0.1) è possibile creare due binari con comportamenti differenti ma con lo stesso MD5, in questo caso **good** e **evil**

In [5]:
!md5sum good

59e240a4ea963693b39cdfed2955f858  good


In [6]:
!md5sum evil

59e240a4ea963693b39cdfed2955f858  evil


Creo due funzioni differenti con due messaggi in output differenti

In [7]:
%%writefile verofalso.c


#include <stdio.h>
#include <unistd.h>

/* do something innocent */
int main_good(int ac, char *av[]) {
  fprintf(stdout, "Un messaggio vero!\n");
  sleep(1);
  return 0;
}

/* do something evil */
int main_evil(int ac, char *av[]) {
  fprintf(stdout, "Un messaggio falso!\n");
  sleep(1);
  return 0;
}

Writing verofalso.c


Compilo

In [0]:
!gcc verofalso.c goodevil.o -o verofalso

Creo due binari, corrispondenti alle due funzioni


In [9]:
!./evilize verofalso -g vero -e falso

Initial vector: 0xc3c11b57 0x18037329 0x21f67871 0xf44dc0f0
Searching for MD5 collision (this can take several hours)...
Random seed: 1580722640
Progress: 1.0Progress: 1.1Progress: 1.2Progress: 1.3Progress: 1.4Progress: 1.5Progress: 1.6Progress: 1.7Progress: 1.8Progress: 1.9Progress: 1.10Progress: 1.11Progress: 1.12Progress: 1.13Progress: 1.14Progress: 1.15Progress: 1.16Progress: 1.17Progress: 1.18Progress: 1.19Progress: 1.20Progress: 1.21Progress: 1.22Progress: 1.23Progress: 1.24Progress: 1.25Progress: 1.26Progress: 1.27Progress: 1.28Progress: 1.29Progress: 1.30Progress: 1.31Progress: 1.32Progress: 1.33Progress: 1.34Progress: 1.35Progress: 1.36Progress: 1.37Progress: 1.38Progress: 1.39Progress: 1.40Progress: 1.41Progress: 1.42Progress: 1.43Progress: 1.44Progress: 1.45Progress: 1.46Progress: 1.47Progress: 1.48Progress: 1.49Progress: 1.50Progress: 1.51Progress: 1.52Progress: 1.53Progress: 1.54Progress: 1.55Progress: 1.56Progress: 1.57Progress: 1.58Progress: 1.59Progress: 1.60Progress: 1

**Verifico i due binari**

Imposto i permessi

In [0]:
!chmod 755 vero
!chmod 755 falso

Verifico il primo

In [14]:
!./vero

Un messaggio vero!


In [10]:
!md5sum vero

d18f76d66e1f1a8cde9797f19d957bc8  vero


Verifico il secondo

In [17]:
!./falso

Un messaggio vero!


In [11]:
!md5sum falso

d18f76d66e1f1a8cde9797f19d957bc8  falso


In [0]:
import hashlib
def md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

In [21]:
if md5('vero') == md5('falso'):
  print('gli hash sono identici, nonostante le funzioni siano differenti, MD5 non è univoco')
else:
  print('gli hash sono diversi')

gli hash sono identici, nonostante le funzioni siano differenti, MD5 non è univoco
